<a href="https://colab.research.google.com/github/mgnarag/binarization-lowres/blob/main/Metric_RGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
!ls drive/My\ Drive
file_path = "/content/drive/My Drive/Architectural_designs/From Arki/"

'107.1 syllabus 1SAY1920 FINAL.gdoc'
'1C-01 Narag et al (1).pdf'
'1C-01 Narag et al.pdf'
'1FA971504AC646559D05A8ED6341AFB3[1025340].png'
'1H-04 Narag et al.pdf'
 20220802_182749.heic
 20220802_182831.heic
 20220802_183259.heic
 20220803_125819.heic
 20220803_125842.heic
 20220803_144229.heic
 20220803_144422.heic
 20220803_160724.heic
 20220803_160751.heic
 20220803_165240.heic
 20220803_165318.heic
 20220803_174701.heic
 20220803_174706.heic
 20220804_175217.heic
 20220804_175443.heic
 20220804_175616.heic
 20220804_181917.heic
 20220804_182007.heic
 20220804_182117.heic
 2S22-23_LE3-Regular-Set-A-FINAL-KEY.pdf
'A4_Physics 71 TWHFU-3_gradesheet (1).pdf'
'A4_Physics 71 TWHFU-3_officialgrades (1).pdf'
'A4_Physics 71 TWHFU-3_officialgrades.gdoc'
'Abstract - NARAG, Mark Jeremy.pdf'
'Annex 1 (copy of Journal).pdf'
'Annex 2 (Proof of employments).pdf'
'App Physics 181 THY-FX-2_studentcontactlist.xlsx'
 Architectural_designs
'Binarizing architectural drawings'
 Classroom
'Colab Notebooks'
'C

In [27]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf


from PIL import Image, ImageOps
Image.MAX_IMAGE_PIXELS = None
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow import keras

import time
import os
import cv2

In [28]:
def crop(im):
    width, height = im.size
    data = []
    for j in range(0,int(height/n_size)):
        for i in range(0,int(width/n_size)):
            im1 = im.crop((0 + (n_size*i), 0 + (n_size*j), n_size + (n_size*i), n_size + (n_size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            im1 = im1/255
            data.append(im1)
    return data


In [29]:
gt = Image.open(file_path+ '4 Testing/ground_truth_100dpi.png').convert('L')
img_gt = cv2.imread(file_path+ '4 Testing/ground_truth_100dpi.png')
gt = np.asarray(gt) > 128
gt = ~gt

filename = "nbs_100dpi_wb.png"
directory = file_path + '4 Testing/' + filename
#directory = 'test_output_v'+str(model)+'.png'
image = Image.open(directory)
rgb = image.convert("RGB")
r, g, b = rgb.split()
w_dirty, h_dirty = r.size

gray = ImageOps.grayscale(Image.open(directory))
gray = np.array(gray)
gray = Image.fromarray(gray.astype('uint8'))


dirty = [gray,r,g,b]




METRIC = []
channel = ['gray','red','green', 'blue']
for model in range(1,7):
  for color in range(0,4):
      print('Model = ', model, '', str(channel[color]))
      from tensorflow import keras
      autoencoder = keras.models.load_model(file_path + '1 Models/100 DPI white balanced and bw output/autoencoder_'+str(model)+ '_'+str(channel[color]))
      if model == 1:
          n_size = 32
      if model == 2:
          n_size = 32
      if model == 3:
          n_size = 64
      if model == 4:
          n_size = 64
      if model == 5:
          n_size = 128
      if model == 6:
          n_size = 128
      if model == 7:
          n_size = 256
      if model == 8:
          n_size = 256

      xx = int(w_dirty/n_size)
      final=[]

      for portion in range(0,xx):
          test = dirty[color]
          im1 = test.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
          w1, h1 = im1.size
          w = int(w1/n_size)
          h = int(h1/n_size)

          neverbeforeseen = np.array(crop(im1))
          encoded_imgs = autoencoder.encoder(neverbeforeseen).numpy()
          decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

          col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
          for i in range(2,h):
              col = np.vstack((col,decoded_imgs[i]))

          y = np.where(col > 0.5,1,0) #round the values
          y = (y * 255).astype('uint8')
          if portion == 0:
              final = y
          if portion > 0:
              final = np.hstack((final,y))

      final = np.squeeze(final)
      reconstructed = Image.fromarray(final)
      reconstructed.save(file_path + "5 Tested/output v" + str(model) + '_'+str(channel[color]) + '.png')

      output = Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[color]) +  '.png').convert('L')  # convert to grayscale
      img_out = cv2.imread(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[color]) +  '.png')

      output = np.asarray(output) > 128 # Convert to binary images
      output = ~output

      tp = np.sum(output & gt)
      fp = np.sum(output & ~gt)
      fn = np.sum(~output & gt)
      precision = tp / (tp + fp)
      recall = tp / (tp + fn)
      f1score = (2*precision*recall)/(precision + recall)

      I_1 = output & gt
      U_1 = output | gt
      I = np.sum(output & gt)
      U = np.sum(output | gt)
      i_o_u = I/U

      psnr = cv2.PSNR(img_out, img_gt)

      METRIC.append([model, str(channel[color]), precision,recall,f1score,i_o_u,psnr])
  #COMBINING RGB ------------------------------------------------
  print('Model = ', model, '', str('RGB'))
  output_red = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[0]) +  '.png').convert('L'))>128)
  output_green = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[1]) +  '.png').convert('L'))>128)
  output_blue = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[2]) +  '.png').convert('L'))>128)
  output_rgb = ~output_red | ~output_green | ~output_blue

  reconstructed = Image.fromarray(~output_rgb)
  reconstructed.save(file_path + "5 Tested/output v" + str(model) + '_RGB.png')

  output = Image.open(file_path + "5 Tested/output v" + str(model)  + '_RGB.png').convert('L')  # convert to grayscale
  img_out = cv2.imread(file_path + "5 Tested/output v" + str(model) + '_RGB.png')

  output = np.asarray(output) > 128
  output = ~output

  tp = np.sum(output & gt)
  fp = np.sum(output & ~gt)
  fn = np.sum(~output & gt)
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1score = (2*precision*recall)/(precision + recall)

  I_1 = output & gt
  O_1 = output | gt
  I = np.sum(output & gt)
  O = np.sum(output | gt)
  i_o_u = I/O

  psnr = cv2.PSNR(img_out, img_gt)

  METRIC.append([model, str('RGB'), precision,recall,f1score,i_o_u,psnr])
  #COMBINING RGB plus grayscale------------------------------------------------
  print('Model = ', model, '', str('RGB+gray'))
  output_gray = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model) +  '_gray.png').convert('L'))>128)
  output = output_rgb | ~output_gray

  reconstructed = Image.fromarray(~output)
  reconstructed.save(file_path + "5 Tested/output v" + str(model) + '_RGB_GRAY.png')

  output = Image.open(file_path + "5 Tested/output v" + str(model)  + '_RGB_GRAY.png').convert('L')  # convert to grayscale
  img_out = cv2.imread(file_path + "5 Tested/output v" + str(model) + '_RGB_GRAY.png')

  output = np.asarray(output) > 128
  output = ~output

  tp = np.sum(output & gt)
  fp = np.sum(output & ~gt)
  fn = np.sum(~output & gt)
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1score = (2*precision*recall)/(precision + recall)

  I_1 = output & gt
  O_1 = output | gt
  I = np.sum(output & gt)
  O = np.sum(output | gt)
  i_o_u = I/O

  psnr = cv2.PSNR(img_out, img_gt)

  METRIC.append([model, str('RGB+gray'),precision,recall,f1score,i_o_u,psnr])


METRIC = pd.DataFrame(METRIC)

headers = ['Model','Channel','Precision', 'Recall', 'F1 Score', 'IOU','PSNR' ]
METRIC.columns = headers

METRIC.to_csv(file_path + '[B] 100 DPI white balanced and bw output.csv')
print(METRIC)

Model =  1  gray
Model =  1  red


<ipython-input-29-f262f0fbcacd>:86: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


Model =  1  green
Model =  1  blue
Model =  1  RGB
Model =  1  RGB+gray
Model =  2  gray
Model =  2  red
Model =  2  green
Model =  2  blue
Model =  2  RGB
Model =  2  RGB+gray
Model =  3  gray
Model =  3  red
Model =  3  green
Model =  3  blue
Model =  3  RGB
Model =  3  RGB+gray
Model =  4  gray
Model =  4  red
Model =  4  green
Model =  4  blue
Model =  4  RGB
Model =  4  RGB+gray
Model =  5  gray
Model =  5  red
Model =  5  green
Model =  5  blue
Model =  5  RGB
Model =  5  RGB+gray
Model =  6  gray
Model =  6  red
Model =  6  green
Model =  6  blue
Model =  6  RGB
Model =  6  RGB+gray
    Model   Channel  Precision    Recall  F1 Score       IOU       PSNR
0       1      gray   0.828627  0.746283  0.785302  0.646500  15.690473
1       1       red        NaN  0.000000       NaN  0.000000  11.797712
2       1     green   0.823728  0.765470  0.793531  0.657731  15.795230
3       1      blue   0.838856  0.688308  0.756162  0.607926  15.324710
4       1       RGB   0.788938  0.821037  0

In [30]:
'''
TP = output & gt
TP = Image.fromarray(~TP)
plt.imshow(TP,cmap = 'gray')
TP.save(file_path + "5 Tested/TP.png")

FP = output & ~gt
FP = Image.fromarray(~FP)
plt.imshow(FP,cmap = 'gray')
FP.save(file_path + "5 Tested/FP.png")

FN = ~output & gt
FN = Image.fromarray(~FN)
plt.imshow(FN,cmap = 'gray')
FN.save(file_path + "5 Tested/FN.png")
'''

'\nTP = output & gt\nTP = Image.fromarray(~TP)\nplt.imshow(TP,cmap = \'gray\')\nTP.save(file_path + "5 Tested/TP.png")\n\nFP = output & ~gt\nFP = Image.fromarray(~FP)\nplt.imshow(FP,cmap = \'gray\')\nFP.save(file_path + "5 Tested/FP.png")\n\nFN = ~output & gt\nFN = Image.fromarray(~FN)\nplt.imshow(FN,cmap = \'gray\')\nFN.save(file_path + "5 Tested/FN.png")\n'